In [ ]:
import json
import pandas as pd
import numpy as np
import os
from unidecode import unidecode # Pour enlever les accents


✅ Fichier de test créé avec succès !


Étape 1 : Chargement et Inspection brute
On charge le JSON et on regarde comment il est structuré. C'est souvent là qu'on comprend pourquoi c'est "complexe".

In [14]:
# 1. Chargement
json_path = 'data/data_2018.json'

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Inspection
print("🔑 Les clés principales du JSON :", data.keys())
print("-" * 30)
print("Exemple d'une équipe brute :", data['teams'][0])
print("Exemple d'un groupe brut :", data['groups']['c']['name'])

🔑 Les clés principales du JSON : dict_keys(['stadiums', 'tvchannels', 'teams', 'groups', 'knockout'])
------------------------------
Exemple d'une équipe brute : {'id': 1, 'name': 'Russia', 'fifaCode': 'RUS', 'iso2': 'ru', 'flag': 'https://upload.wikimedia.org/wikipedia/en/thumb/f/f3/Flag_of_Russia.svg/900px-Flag_of_Russia.png', 'emoji': 'flag-ru', 'emojiString': '🇷🇺'}
Exemple d'un groupe brut : Group C


In [23]:
# Juste pour info, voici ce qu'il y a dedans
stadiums = pd.DataFrame(data['stadiums'])
display(stadiums.head())

,id,name,city,lat,lng,image
0,1,Luzhniki Stadium,Moscow,55.715765,37.551522,https://upload.wikimedia.org/wikipedia/commons...
1,2,Otkrytiye Arena,Moscow,55.817765,37.440363,https://upload.wikimedia.org/wikipedia/commons...
2,3,Krestovsky Stadium,Saint Petersburg,59.972740,30.221408,https://upload.wikimedia.org/wikipedia/commons...
3,4,Kaliningrad Stadium,Kaliningrad,54.698157,20.533859,https://upload.wikimedia.org/wikipedia/commons...
4,5,Kazan Arena,Kazan,55.820983,49.160966,https://upload.wikimedia.org/wikipedia/commons...


In [24]:
# Juste pour info, voici ce qu'il y a dedans
teams = pd.DataFrame(data['teams'])
display(teams.head())

,id,name,fifaCode,iso2,flag,emoji,emojiString
0,1,Russia,RUS,ru,https://upload.wikimedia.org/wikipedia/en/thum...,flag-ru,🇷🇺
1,2,Saudi Arabia,KSA,sa,https://upload.wikimedia.org/wikipedia/commons...,flag-sa,🇸🇦
2,3,Egypt,EGY,eg,https://upload.wikimedia.org/wikipedia/commons...,flag-eg,🇪🇬
3,4,Uruguay,URU,uy,https://upload.wikimedia.org/wikipedia/commons...,flag-uy,🇺🇾
4,5,Portugal,POR,pt,https://upload.wikimedia.org/wikipedia/commons...,flag-pt,🇵🇹


In [25]:
# Juste pour info, voici ce qu'il y a dedans
groups = pd.DataFrame(data['groups'])
display(groups.head())

,a,b,c,d,e,f,g,h
name,Group A,Group B,Group C,Group D,Group E,Group F,Group G,Group H
winner,4,6,9,15,17,23,25,31
runnerup,1,5,12,13,18,22,28,32
matches,"[{'name': 1, 'type': 'group', 'home_team': 1, ...","[{'name': 3, 'type': 'group', 'home_team': 5, ...","[{'name': 5, 'type': 'group', 'home_team': 9, ...","[{'name': 7, 'type': 'group', 'home_team': 13,...","[{'name': 9, 'type': 'group', 'home_team': 17,...","[{'name': 11, 'type': 'group', 'home_team': 21...","[{'name': 13, 'type': 'group', 'home_team': 25...","[{'name': 15, 'type': 'group', 'home_team': 29..."


In [26]:
# Juste pour info, voici ce qu'il y a dedans
tv_channels = pd.DataFrame(data['tvchannels'])
display(tv_channels.head())

,id,name,icon,country,iso2,lang
0,1,DR1,https://upload.wikimedia.org/wikipedia/commons...,Denmark,dk,[da]
1,2,TV2,https://upload.wikimedia.org/wikipedia/commons...,Denmark,dk,[da]
2,3,BBC UK,https://upload.wikimedia.org/wikipedia/commons...,UK,en,[en]
3,4,ITV UK,https://upload.wikimedia.org/wikipedia/en/9/92...,UK,en,[en]
4,5,ITV4 UK,https://upload.wikimedia.org/wikipedia/en/9/92...,UK,en,[en]


1. La règle du "Besoin Métier"

Dans un ETL, on ne garde pas tout. On garde seulement ce qui est utile pour l'analyse finale.
L'objectif du projet : Analyser les résultats, les scores et prédire les vainqueurs.

La question : Est-ce que savoir que le match a été diffusé sur BBC UK ou TF1 nous aide à savoir qui a gagné ? Non.
C'est ce qu'on appelle la sélection de données. Stocker des informations inutiles (comme les chaînes TV) alourdit la base de données et ralentit les calculs pour rien.

2. Les clés JSON : Gardé ou pas et	Pourquoi ?

teams	✅ Gardé	Indispensable pour avoir le nom des pays (Lookup).

stadiums	✅ Gardé	Utile pour avoir la Ville (city) demandée dans le livrable.

groups	✅ Gardé	Contient les matchs de poule (la donnée principale).

knockout	✅ Gardé	Contient les finales (la donnée principale).

tvchannels	❌ Rejeté	Information "cosmétique" sans impact sur le score du match.

In [27]:
# Regardons seulement les matchs du Groupe A (colonne 'a')
matches_groupe_a = data['groups']['a']['matches']

# Affichons le premier match de la liste
premier_match = matches_groupe_a[0]

print("🏟️ Voici à quoi ressemble UN match caché dans cette liste :")
print(premier_match)

🏟️ Voici à quoi ressemble UN match caché dans cette liste :
{'name': 1, 'type': 'group', 'home_team': 1, 'away_team': 2, 'home_result': 5, 'away_result': 0, 'date': '2018-06-14T18:00:00+03:00', 'stadium': 1, 'channels': [4, 6, 13, 17, 20, 22], 'finished': True, 'matchday': 1}


Étape 2 : Création des "Lookups" (Dictionnaires de correspondance)
On transforme les listes en dictionnaires pour pouvoir traduire les ID instantanément.

In [16]:
# MAPPING ÉQUIPES
# On veut : {9: "France", 15: "Croatia"}
teams_map = {t['id']: t['name'] for t in data['teams']}

print("🗺️ Dictionnaire des équipes (Extrait) :")
print(teams_map)

# MAPPING STADES (Pour avoir la Ville)
# On veut : {1: "Moscow"}
stadiums_map = {s['id']: s['city'] for s in data['stadiums']}

print("\n🏟️ Dictionnaire des villes (Extrait) :")
print(stadiums_map)

# Test manuel pour vérifier
print("-" * 30)
print(f"L'équipe ID 9 est : {teams_map.get(9)}")
print(f"Le stade ID 1 est à : {stadiums_map.get(1)}")

🗺️ Dictionnaire des équipes (Extrait) :
{1: 'Russia', 2: 'Saudi Arabia', 3: 'Egypt', 4: 'Uruguay', 5: 'Portugal', 6: 'Spain', 7: 'Morocco', 8: 'Iran', 9: 'France', 10: 'Australia', 11: 'Peru', 12: 'Denmark', 13: 'Argentina', 14: 'Iceland', 15: 'Croatia', 16: 'Nigeria', 17: 'Brazil', 18: 'Switzerland', 19: 'Costa Rica', 20: 'Serbia', 21: 'Germany', 22: 'Mexico', 23: 'Sweden', 24: 'South Korea', 25: 'Belgium', 26: 'Panama', 27: 'Tunisia', 28: 'England', 29: 'Poland', 30: 'Senegal', 31: 'Colombia', 32: 'Japan'}

🏟️ Dictionnaire des villes (Extrait) :
{1: 'Moscow', 2: 'Moscow', 3: 'Saint Petersburg', 4: 'Kaliningrad', 5: 'Kazan', 6: 'Nizhny Novgorod', 7: 'Samara', 8: 'Volgograd', 9: 'Saransk', 10: 'Rostov-on-Don', 11: 'Sochi', 12: 'Yekaterinburg'}
------------------------------
L'équipe ID 9 est : France
Le stade ID 1 est à : Moscow


Étape 3 : Boucle sur les Matchs de Poules (Groups)
On rentre dans la structure imbriquée groups -> a, b, c... -> matches.

In [17]:
matches_list = []

print("🔄 Début de l'extraction des Groupes...")

# data['groups'] est un dictionnaire (clés "a", "b", "c"...)
for group_letter, group_content in data['groups'].items():
    
    nom_groupe = group_content['name'] # ex: "Group C"
    liste_matchs = group_content['matches']
    
    print(f"   -> Traitement du {nom_groupe} ({len(liste_matchs)} matchs)")
    
    for m in liste_matchs:
        # C'est ICI qu'on fait la traduction ID -> Nom
        match_propre = {
            'home_team': teams_map.get(m['home_team']), # Traduction !
            'away_team': teams_map.get(m['away_team']), # Traduction !
            'home_result': m['home_result'],
            'away_result': m['away_result'],
            'date': m['date'],
            'city': stadiums_map.get(m['stadium']),     # Traduction !
            'round': nom_groupe,
            'edition': 2018
        }
        matches_list.append(match_propre)

print(f"✅ Nombre de matchs récupérés : {len(matches_list)}")
print("Dernier match ajouté :", matches_list[-1])

🔄 Début de l'extraction des Groupes...
   -> Traitement du Group A (6 matchs)
   -> Traitement du Group B (6 matchs)
   -> Traitement du Group C (6 matchs)
   -> Traitement du Group D (6 matchs)
   -> Traitement du Group E (6 matchs)
   -> Traitement du Group F (6 matchs)
   -> Traitement du Group G (6 matchs)
   -> Traitement du Group H (6 matchs)
✅ Nombre de matchs récupérés : 48
Dernier match ajouté : {'home_team': 'Senegal', 'away_team': 'Colombia', 'home_result': 0, 'away_result': 1, 'date': '2018-06-28T18:00:00+04:00', 'city': 'Samara', 'round': 'Group H', 'edition': 2018}


Étape 4 : Boucle sur les Phases Finales (Knockout)
Même logique, mais la clé s'appelle knockout.

In [18]:
print("🔄 Début de l'extraction des Phases Finales...")

# data['knockout'] contient "round_16", "round_8", "round_2" (finale), etc.
for stage_key, stage_content in data['knockout'].items():
    
    nom_phase = stage_content['name'] # ex: "Final"
    liste_matchs = stage_content['matches']
    
    print(f"   -> Traitement de : {nom_phase}")
    
    for m in liste_matchs:
        match_propre = {
            'home_team': teams_map.get(m['home_team']),
            'away_team': teams_map.get(m['away_team']),
            'home_result': m['home_result'],
            'away_result': m['away_result'],
            'date': m['date'],
            'city': stadiums_map.get(m['stadium']),
            'round': nom_phase,
            'edition': 2018
        }
        matches_list.append(match_propre)

print(f"✅ Total cumulé des matchs : {len(matches_list)}")

🔄 Début de l'extraction des Phases Finales...
   -> Traitement de : Round of 16
   -> Traitement de : Quarter-finals
   -> Traitement de : Semi-finals
   -> Traitement de : Third place play-off
   -> Traitement de : Final
✅ Total cumulé des matchs : 64


Étape 5 : Création du DataFrame et Nettoyage Final
On transforme la liste de dictionnaires en tableau propre.

In [20]:
# Conversion en DataFrame Pandas
df = pd.DataFrame(matches_list)

print("📊 DataFrame Brut (Regarde la colonne date) :")
display(df.head()) # 'display' fonctionne dans les notebooks, sinon utilise print()

# Nettoyage de la date
# Format actuel : "2018-06-16T13:00:00+03:00" (Trop compliqué)
# Format voulu : "2018-06-16" (Date simple)



📊 DataFrame Brut (Regarde la colonne date) :


,home_team,away_team,home_result,away_result,date,city,round,edition
0,Russia,Saudi Arabia,5,0,2018-06-14T18:00:00+03:00,Moscow,Group A,2018
1,Egypt,Uruguay,0,1,2018-06-15T17:00:00+05:00,Yekaterinburg,Group A,2018
2,Russia,Egypt,3,1,2018-06-19T21:00:00+03:00,Saint Petersburg,Group A,2018
3,Uruguay,Saudi Arabia,1,0,2018-06-20T18:00:00+03:00,Rostov-on-Don,Group A,2018
4,Uruguay,Russia,3,0,2018-06-25T18:00:00+04:00,Samara,Group A,2018


In [30]:
import json
import pandas as pd

# 1. Chargement du fichier
with open('data/data_2018.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Extraction directe
# La clé 'teams' contient déjà une liste parfaite, Pandas peut la lire directement
df_countries = pd.DataFrame(data['teams'])

# 3. Sélection des colonnes utiles pour ton projet
# On ne garde pas 'emoji' ou 'emojiString', c'est inutile pour la base de données
df_countries = df_countries[['id', 'name', 'fifaCode', 'iso2', 'flag']]

# 4. Renommage pour que ce soit propre en base (Optionnel)
df_countries = df_countries.rename(columns={
    'id': 'team_id',
    'name': 'team_name',
    'fifaCode': 'fifa_code',
    'iso2': 'iso_code'
})

print(f"🌍 Nombre de pays qualifiés en 2018 : {len(df_countries)}")
display(df_countries)

🌍 Nombre de pays qualifiés en 2018 : 32


,team_id,team_name,fifa_code,iso_code,flag
0,1,Russia,RUS,ru,https://upload.wikimedia.org/wikipedia/en/thum...
1,2,Saudi Arabia,KSA,sa,https://upload.wikimedia.org/wikipedia/commons...
2,3,Egypt,EGY,eg,https://upload.wikimedia.org/wikipedia/commons...
3,4,Uruguay,URU,uy,https://upload.wikimedia.org/wikipedia/commons...
4,5,Portugal,POR,pt,https://upload.wikimedia.org/wikipedia/commons...
5,6,Spain,ESP,es,https://upload.wikimedia.org/wikipedia/en/thum...
6,7,Morocco,MAR,ma,https://upload.wikimedia.org/wikipedia/commons...
7,8,Iran,IRN,ir,https://upload.wikimedia.org/wikipedia/commons...
8,9,France,FRA,fr,https://upload.wikimedia.org/wikipedia/en/thum...
9,10,Australia,AUS,au,https://upload.wikimedia.org/wikipedia/commons...


Vérification "Qualité de données"

In [28]:
# Vérifier s'il y a des valeurs manquantes (ex: un ID d'équipe qu'on n'aurait pas trouvé dans le mapping)
print("🔍 Vérification des valeurs nulles :")
print(df.isnull().sum())

if df.isnull().values.any():
    print("⚠️ ATTENTION : Il y a des données manquantes (peut-être un ID inconnu dans le JSON ?)")
else:
    print("✅ Tout est propre !")

🔍 Vérification des valeurs nulles :
home_team      0
away_team      0
home_result    0
away_result    0
date           0
city           0
round          0
edition        0
dtype: int64
✅ Tout est propre !


In [31]:
# On change les options d'affichage de Pandas pour voir illimité
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(f"📋 Affichage complet des {len(df)} matchs :")
display(df) 
# Si display() ne marche pas (ex: hors Jupyter), utilise : print(df.to_string())

📋 Affichage complet des 64 matchs :


,home_team,away_team,home_result,away_result,date,city,round,edition
0,Russia,Saudi Arabia,5,0,2018-06-14T18:00:00+03:00,Moscow,Group A,2018
1,Egypt,Uruguay,0,1,2018-06-15T17:00:00+05:00,Yekaterinburg,Group A,2018
2,Russia,Egypt,3,1,2018-06-19T21:00:00+03:00,Saint Petersburg,Group A,2018
3,Uruguay,Saudi Arabia,1,0,2018-06-20T18:00:00+03:00,Rostov-on-Don,Group A,2018
4,Uruguay,Russia,3,0,2018-06-25T18:00:00+04:00,Samara,Group A,2018
5,Saudi Arabia,Egypt,2,1,2018-06-25T17:00:00+03:00,Volgograd,Group A,2018
6,Portugal,Spain,3,3,2018-06-15T21:00:00+03:00,Sochi,Group B,2018
7,Morocco,Iran,0,1,2018-06-15T18:00:00+03:00,Saint Petersburg,Group B,2018
8,Portugal,Morocco,1,0,2018-06-20T15:00:00+03:00,Moscow,Group B,2018
9,Iran,Spain,0,1,2018-06-20T21:00:00+03:00,Kazan,Group B,2018
